In [1]:
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path

from biopsykit.classification.model_selection import SklearnPipelinePermuter

from pepbench.io import compute_abs_error

from pepbench.plotting.results import boxplot_algorithm_performance, _plot_helper_algorithm_performance

from fau_colors import cmaps, register_fausans_font

In [2]:
#register_fausans_font()
plt.close("all")

palette = sns.color_palette(cmaps.faculties)
sns.set_theme(context="notebook", style="ticks", font="sans-serif", palette=palette)

plt.rcParams["figure.figsize"] = (10, 5)
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["mathtext.default"] = "regular"
plt.rcParams["font.family"] = "sans-serif"
#plt.rcParams["font.sans-serif"] = "FAUSans Office"

palette

[(0.0, 0.1843137254901961, 0.4235294117647059),
 (0.4666666666666667, 0.6235294117647059, 0.7098039215686275),
 (1.0, 0.7215686274509804, 0.10980392156862745),
 (0.0, 0.6392156862745098, 0.8784313725490196),
 (0.2627450980392157, 0.6901960784313725, 0.16470588235294117),
 (0.7843137254901961, 0.06274509803921569, 0.1803921568627451)]

In [3]:
data_path = Path("../../../results/data/q-peak")
model_path = Path("../../../results/models/q-peak")
output_image_path = Path("Images/Paper")

In [4]:
merged_ml_results_q_peak_metrics = pd.read_csv(data_path.joinpath("merged_ml_results_q_peak_error_metrics.csv"), index_col=[0,1,2,3,4])
merged_ml_results_q_peak_metrics

forounzafar2018_rel_error  \
participant condition phase         heartbeat_id_reference forounzafar2018_error                              
GDN0005     Dummy     HoldingBreath 0                      -16.0                                  -0.061538   
                                    1                      -18.0                                  -0.069231   
                                    3                      -14.0                                  -0.063063   
                                    4                      -16.0                                  -0.072727   
                                    5                      -14.0                                  -0.061404   
...                                                                                                     ...   
VP_032      tsst      Talk          39                     -11.0                                  -0.063953   
                                    40                     -12.0                                  -0.066298   
                                    41                     -10.0                                  -0.050000   
                                    42                     -11.0                                  -0.051887   
                                    43                     -10.0                                  -0.048309   

                                                                                  forounzafar2018_abs_error  \
participant condition phase         heartbeat_id_reference forounzafar2018_error                              
GDN0005     Dummy     HoldingBreath 0                      -16.0                                       16.0   
                                    1                      -18.0                                       18.0   
                                    3                      -14.0                                       14.0   
                                    4                      -16.0                                       16.0   
                                    5                      -14.0                                       14.0   
...                                                                                                     ...   
VP_032      tsst      Talk          39                     -11.0                                       11.0   
                                    40                     -12.0                                       12.0   
                                    41                     -10.0                                       10.0   
                                    42                     -11.0                                       11.0   
                                    43                     -10.0                                       10.0   

                                                                                  forounzafar2018_abs_rel_error  \
participant condition phase         heartbeat_id_reference forounzafar2018_error                                  
GDN0005     Dummy     HoldingBreath 0                      -16.0                                       0.061538   
                                    1                      -18.0                                       0.069231   
                                    3                      -14.0                                       0.063063   
                                    4                      -16.0                                       0.072727   
                                    5                      -14.0                                       0.061404   
...                                                                                                         ...   
VP_032      tsst      Talk          39                     -11.0                                       0.063953   
                                    40                     -12.0                                       0.066298   
                                    41      

In [5]:
merged_ml_results_q_peak = pd.read_csv(data_path.joinpath("merged_ml_results_q_peak.csv"), index_col=[0,1,2,3]).drop(columns=['level_0'])
merged_ml_results_q_peak

q_peak_sample_reference  \
participant condition phase         heartbeat_id_reference                            
GDN0005     Dummy     HoldingBreath 0                                         260.0   
                                    1                                         260.0   
                                    3                                         222.0   
                                    4                                         220.0   
                                    5                                         228.0   
...                                                                             ...   
VP_032      tsst      Talk          39                                        172.0   
                                    40                                        181.0   
                                    41                                        200.0   
                                    42                                        212.0   
                                    43                                        207.0   

                                                            rr_interval_ms_estimated  \
participant condition phase         heartbeat_id_reference                             
GDN0005     Dummy     HoldingBreath 0                                          850.0   
                                    1                                          778.0   
                                    3                                          746.0   
                                    4                                          766.0   
                                    5                                          790.0   
...                                                                              ...   
VP_032      tsst      Talk          39                                         619.0   
                                    40                                         680.0   
                                    41                                         719.0   
                                    42                                         698.0   
                                    43                                         689.0   

                                                            forounzafar2018  \
participant condition phase         heartbeat_id_reference                    
GDN0005     Dummy     HoldingBreath 0                                 276.0   
                                    1                                 278.0   
                                    3                                 236.0   
                                    4                                 236.0   
                                    5                                 242.0   
...                                                                     ...   
VP_032      tsst      Talk          39                                183.0   
                                    40                                193.0   
                                    41                                210.0   
                                    42                                223.0   
                                    43                                217.0   

                                                            martinez2004  \
participant condition phase         heartbeat_id_reference                 
GDN0005     Dummy     HoldingBreath 0                              262.0   
                                    1                              262.0   
                                    3                              224.0   
                                    4                              222.0   
                                    5                              230.0   
...                                                                  ...   
VP_032      tsst      Talk          39                             173.0   
                                    40                             182.

### Best performing ML-Estimators
- ('StandardScaler', 'SelectKBest', 'SVR') Data: RR-Interval-Median-Imputed
- ('MinMaxScaler', 'SelectKBest', 'RandomForestRegressor') Data: RR-Interval-Include-Nan
- ('StandardScaler', 'SelectKBest', 'RandomForestRegressor') Data: RR-Interval-Median-Imputed